In [1]:
import autograd
from autograd import numpy as np

In [2]:
from collections import OrderedDict
import paragami

In [3]:
import time

a = np.full(100000, float('nan'))
b = np.random.random(a.shape)

def set_inplace(b, a):
    a[:] = b + 2
    return a

def set_notinplace(b):
    return b + 2

set_inplace(b, a)

t1 = time.time()
for iter in range(1000):
    set_inplace(b, a)
print('Inplace: ', time.time() - t1)

t1 = time.time()
for iter in range(1000):
    a = set_notinplace(b)
print('Not in place: ', time.time() - t1)

# WTF?  Try %memit and id

Inplace:  0.10454082489013672
Not in place:  0.035413503646850586


In [4]:


def check_dict_equal(dict1, dict2):
    assert dict1.keys() == dict2.keys()
    for key  in dict1:
        assert_array_almost_equal(dict1[key], dict2[key])

dict_pattern = paragami.PatternDict()
dict_pattern['a'] = paragami.NumericArrayPattern((2, 3, 4), lb=-1, ub=2)
dict_pattern['b'] = paragami.NumericArrayPattern((5, ), lb=-1, ub=10)
dict_pattern['c'] = paragami.NumericArrayPattern((5, 2), lb=-1, ub=10)

dict_val = dict_pattern.random()
print(dict_val)
dict_val2 = dict(dict_val)
print(dict_val2)
flat_val1 = dict_pattern.flatten(dict_val, free=True)
flat_val2 = dict_pattern.flatten(dict_val2, free=True)

np.linalg.norm(flat_val1 - flat_val2)

OrderedDict([('a', array([[[0.79043707, 1.04347342, 0.68497739, 0.69084469],
        [1.10278198, 0.50384999, 0.99964562, 0.75926731],
        [1.18862299, 0.81566452, 0.87612792, 0.81300401]],

       [[0.77402682, 0.64949695, 0.9560174 , 1.11219744],
        [0.50642725, 1.1576952 , 0.73855495, 0.84763819],
        [1.01231051, 1.02041714, 1.1214629 , 0.67211588]]])), ('b', array([6.12093421, 5.88235636, 4.74424065, 6.28812774, 5.3405505 ])), ('c', array([[6.15680847, 6.10963477],
       [6.82339132, 5.86314506],
       [5.63193196, 6.95458761],
       [4.62336481, 4.7146537 ],
       [6.42306584, 6.98169999]]))])
{'a': array([[[0.79043707, 1.04347342, 0.68497739, 0.69084469],
        [1.10278198, 0.50384999, 0.99964562, 0.75926731],
        [1.18862299, 0.81566452, 0.87612792, 0.81300401]],

       [[0.77402682, 0.64949695, 0.9560174 , 1.11219744],
        [0.50642725, 1.1576952 , 0.73855495, 0.84763819],
        [1.01231051, 1.02041714, 1.1214629 , 0.67211588]]]), 'c': array([[6.15

0.0

In [5]:
param_dict = dict_pattern.random()

def test_function(param_dict):
    a = param_dict['a']
    b = param_dict['b']
    return np.mean(a ** 2) + np.mean(b ** 2)

test_function(param_dict)

27.3937713568762

In [6]:
class PatternedFunction:
    def __init__(self, original_fun, pattern, free, argnum=0):
        self._fun = original_fun
        self._argnum = argnum
        self.free = free
        self._pattern = pattern
        
    def __str__(self):
        return('Function: {}\nargnum: {}\nfree: {}\npattern: {}'.format(
            self._fun, self._argnum, self.free, self._pattern))
    
    def __call__(self, *args, **kwargs):
        flat_val = args[self._argnum]
        folded_val = self._pattern.fold(flat_val, free=self.free)
        new_args = args[0:self._argnum] + (folded_val, ) + args[self._argnum + 1:-1]
        return self._fun(*new_args, **kwargs)

    
patterned_test_function = PatternedFunction(test_function, dict_pattern, True)
print(patterned_test_function)

flat_param_dict = dict_pattern.flatten(param_dict, free=True)
assert(np.abs(patterned_test_function(flat_param_dict) - test_function(param_dict)) < 1e-12)

patterned_test_function_grad = autograd.grad(patterned_test_function)
patterned_test_function_grad(flat_param_dict)

patterned_test_function_hessian = autograd.hessian(patterned_test_function)
patterned_test_function_hessian(flat_param_dict)

Function: <function test_function at 0x7fd4c16326a8>
argnum: 0
free: True
pattern: OrderedDict:
	[a] = Array (2, 3, 4) (lb=-1, ub=2)
	[b] = Array (5,) (lb=-1, ub=10)
	[c] = Array (5, 2) (lb=-1, ub=10)
offset:  0
pattern_flat_length:  24
flat_val:  [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan]
slice:  [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
folded val:  OrderedDict([('a', array([[[0.88155779, 0.79940683, 1.18612433, 0.830613  ],
        [1.19155676, 0.91036553, 0.98209019, 0.95025354],
        [0.76489512, 0.73990216, 0.56063993, 0.53210509]],

       [[0.63273558, 1.11654182, 0.89576839, 0.97409594],
        [0.66704146, 0.99945284, 0.54348877, 0.82042651],
        [1.18566842, 1.01460652, 1.04104657, 0.55582713]]])), ('b', array([4.96675412, 4.60192683, 6.46904209, 4.71536712, 4.80431607])), ('c', array([[4.8054842

array([[0.02789679, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03363873, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00249712, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [7]:
class FlattenedFunction:
    def __init__(self, original_fun, patterns, free, argnums=None):
        self._fun = original_fun
        self._patterns = np.atleast_1d(patterns)
        if argnums is None:
            argnums = np.arange(0, len(self._patterns))
        if len(self._patterns.shape) != 1:
            raise ValueError('patterns must be a 1d vector.')
        self._argnums = np.atleast_1d(argnums)
        self._argnum_sort = np.argsort(self._argnums)
        self.free = np.broadcast_to(free, self._patterns.shape)

        self._validate_args()
        
    def _validate_args(self):
        if len(self._argnums.shape) != 1:
            raise ValueError('argnums must be a 1d vector.')
        if len(self._argnums) != len(self._patterns):
            raise ValueError('argnums must be the same length as patterns.')
        if len(self.free.shape) != 1:
            raise ValueError('free must be a single boolean or a 1d vector of booleans.')
        if len(self.free) != len(self._patterns):
            raise ValueError('free must broadcast to the same shape as patterns.')
        
    def __str__(self):
        return('Function: {}\nargnums: {}\nfree: {}\npatterns: {}'.format(
            self._fun, self._argnums, self.free, self._patterns))
    
    def __call__(self, *args, **kwargs):
        # Loop through the arguments from beginning to end, replacing parameters
        # with their flattened values.
        new_args = ()
        last_argnum = 0
        for i in self._argnum_sort:
            argnum = self._argnums[i]
            folded_val = self._patterns[i].fold(args[argnum], free=self.free[i])
            new_args += args[last_argnum:argnum] + (folded_val, )
            last_argnum = argnum + 1
            
        return self._fun(*new_args, **kwargs)


def test_function(x, a, y, b):
    return x ** 2 + y ** 2 + np.mean(a ** 2) + np.mean(b ** 2)


x = 2
y = 3
a = param_dict['a']
b = param_dict['b']
a_flat = dict_pattern['a'].flatten(a, True)
b_flat = dict_pattern['b'].flatten(b, True)


patterns = [dict_pattern['a'], dict_pattern['b']]
flat_test_function = FlattenedFunction(
    test_function, patterns, free=[True, True], argnums=[1, 3])

assert(np.linalg.norm(
    test_function(x, a, y, b) - flat_test_function(x, a_flat, y, b_flat) < 1e-9))

print(flat_test_function)




def test_function(param_dict):
    a = param_dict['a']
    b = param_dict['b']
    return np.mean(a ** 2) + np.mean(b ** 2)

flat_test_function = FlattenedFunction(
    test_function, dict_pattern, free=True)
param_dict_flat = dict_pattern.flatten(param_dict, free=True)
assert(np.linalg.norm(
    test_function(param_dict) - flat_test_function(param_dict_flat) < 1e-9))



Function: <function test_function at 0x7fd4fd0e9950>
argnums: [1 3]
free: [ True  True]
patterns: [<paragami.numeric_array_patterns.NumericArrayPattern object at 0x7fd4c164a2b0>
offset:  0
pattern_flat_length:  24
flat_val:  [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan]
slice:  [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
folded val:  OrderedDict([('a', array([[[0.88155779, 0.79940683, 1.18612433, 0.830613  ],
        [1.19155676, 0.91036553, 0.98209019, 0.95025354],
        [0.76489512, 0.73990216, 0.56063993, 0.53210509]],

       [[0.63273558, 1.11654182, 0.89576839, 0.97409594],
        [0.66704146, 0.99945284, 0.54348877, 0.82042651],
        [1.18566842, 1.01460652, 1.04104657, 0.55582713]]])), ('b', array([4.96675412, 4.60192683, 6.46904209, 4.71536712, 4.80431607])), ('c', array([[4.80548426, 4.77376924],
       

In [8]:
import datetime
np.array([datetime.datetime.now()])

array([array(datetime.datetime(2018, 10, 25, 20, 14, 2, 556674), dtype=object)],
      dtype=object)

In [9]:
a = np.random.random((2, 2))
print('a: ', a)

shape = (3, 4)
foo = np.empty(shape, dtype='object')
for i in range(3):
    for j in range(4):
        foo[i, j] = a
print(foo.shape)
print(foo[0, 1])

a:  [[0.06203855 0.20978307]
 [0.96475861 0.36313578]]
(3, 4)
[[0.06203855 0.20978307]
 [0.96475861 0.36313578]]


In [10]:
a = np.random.random((2, 2))
print(a)
shape = (3, 4)
#foo = np.reshape(np.array([a for i in range(np.prod(shape))]), shape + a.shape)
foo = np.array([a for i in range(np.prod(shape))])
print(foo.shape)
print(foo.dtype)
print('With shape broadcasting:\n', foo[0, 0])

print('Of a dict:')
a = dict(a=5)
shape = (3, 4)
foo = np.reshape(np.array([a for i in range(np.prod(shape))], dtype='object'), shape)
print(foo.shape)
print(foo[0, 0])


print('\nOf an object array:')
a = np.array(np.random.random((2, 2)), dtype='object')
print(a)
shape = (3, 4)
foo = np.reshape(np.array([a for i in range(np.prod(shape))]), shape + a.shape)
print(foo.shape)
print(foo.dtype)


[[0.87781799 0.30976016]
 [0.49649966 0.13139181]]
(12, 2, 2)
float64
With shape broadcasting:
 [0.87781799 0.30976016]
Of a dict:
(3, 4)
{'a': 5}

Of an object array:
[[0.6971065566333948 0.5362039946229162]
 [0.1192441229021296 0.7861633596200103]]
(3, 4, 2, 2)
object


In [11]:
def is_np(a):
    return(type(a) is np.ndarray)

print(is_np(np.random.random((2, 2))))

a = np.array(np.random.random((2, 2)), dtype='object')
print(is_np(a))

print(is_np(dict(a=4)))


True
True
False


In [12]:
# Array
from paragami.base_patterns import PatternArray

pattern_array = PatternArray(
    (2, 3), paragami.NumericArrayPattern(shape=(2, ), lb=-1, ub=10.0))

val = pattern_array.random()
pattern_array.flatten(val, True)
pattern_array.flatten(val, False)



array([6.4249217 , 5.84936888, 6.68453635, 5.45053115, 5.52282452,
       6.49561241, 6.53200954, 5.75059036, 5.87734821, 4.59970054,
       6.43631276, 5.31289551])

In [14]:
import numpy as onp

(2, 3, ) + ()
foo = OrderedDict(a=5)
bar = onp.array([foo for i in range(3)])
print(bar[0])
# OrderedDict([('a', 5)])

print(type(bar[0]))
# <class 'numpy.ndarray'>

print(bar.dtype)
# object

# print(bar[0].shape)
# # ()

# print(bar[0]['a'])
# # IndexError

# print(bar[0][0])
# IndexError: too many indices for array


OrderedDict([('a', 5)])
<class 'collections.OrderedDict'>
object


In [16]:

if False:
    # won't support
    dict_pattern = paragami.PatternDict()
    dict_pattern['a'] = paragami.NumericArrayPattern(shape=(2, ), lb=-2, ub=10.0)
    pattern_array = PatternArray((2, 3), dict_pattern)

    val = pattern_array.random()
    print(val.shape)
    print(type(val[0, 0]))
    print(val[0, 0])

    # __array_ranges = [ range(0, t) for t in (2, 3) ]

    # print(__array_ranges)
    # pattern_array.flatten(val, True)
    #pattern_array.flatten(val, False)


In [23]:
# Array
pattern_array = PatternArray((2, 3), paragami.PDMatrixPattern(size=2))

val = pattern_array.random()
pattern_array.flatten(val, True)
pattern_array.flatten(val, False)



array([1.50890609, 0.01991117, 3.12312281, 2.08792447, 0.55821873,
       1.62644939, 1.08958171, 0.81940601, 1.86515494, 4.35181299,
       0.37834517, 1.26561289, 1.97934086, 0.37626516, 1.52952511,
       6.87670787, 1.17927306, 2.22321554])

In [29]:
type(pattern_array) is PatternArray
print(PatternArray)

<class 'paragami.base_patterns.PatternArray'>


In [30]:
foo = np.random.random((4, 2))
np.sum(foo, axis=0)

array([3.24789042, 1.65392137])